# Use examples of [edges](https://github.com/romainsacchi/edges)

Author: [romainsacchi](https://github.com/romainsacchi)

This notebook shows examples on how to use `edge` to use location-specific
characterization factors in the characterization matrix of `bw2calc`.

## Requirements

* **Pyhton 3.10 or higher (up to 3.11) is highly recommended**
* `bw2calc >= 2.0.0`

# Use case with [brightway2](https://brightway.dev/)

`brightway2` is an open source LCA framework for Python.
To use `premise` from `brightway2`, it requires that you have an activated `brightway2` project with a `biosphere3` database as well as an [ecoinvent](https://ecoinvent.prg) v.3 cut-off or consequential database registered in that project. Please refer to the brightway [documentation](https://brightway.dev) if you do not know how to create a project and install ecoinvent.

In [1]:
from edges import SpatialLCA, get_available_methods
import bw2data

### List of available methods


In [2]:
get_available_methods()

[('AWARE 1.2c', 'Country', 'irri', 'apr'),
 ('AWARE 1.2c', 'Country', 'irri', 'aug'),
 ('AWARE 1.2c', 'Country', 'irri', 'dec'),
 ('AWARE 1.2c', 'Country', 'irri', 'feb'),
 ('AWARE 1.2c', 'Country', 'irri', 'jan'),
 ('AWARE 1.2c', 'Country', 'irri', 'jul'),
 ('AWARE 1.2c', 'Country', 'irri', 'jun'),
 ('AWARE 1.2c', 'Country', 'irri', 'mar'),
 ('AWARE 1.2c', 'Country', 'irri', 'may'),
 ('AWARE 1.2c', 'Country', 'irri', 'nov'),
 ('AWARE 1.2c', 'Country', 'irri', 'oct'),
 ('AWARE 1.2c', 'Country', 'irri', 'sep'),
 ('AWARE 1.2c', 'Country', 'irri', 'yearly'),
 ('AWARE 1.2c', 'Country', 'non', 'irri', 'apr'),
 ('AWARE 1.2c', 'Country', 'non', 'irri', 'aug'),
 ('AWARE 1.2c', 'Country', 'non', 'irri', 'dec'),
 ('AWARE 1.2c', 'Country', 'non', 'irri', 'feb'),
 ('AWARE 1.2c', 'Country', 'non', 'irri', 'jan'),
 ('AWARE 1.2c', 'Country', 'non', 'irri', 'jul'),
 ('AWARE 1.2c', 'Country', 'non', 'irri', 'jun'),
 ('AWARE 1.2c', 'Country', 'non', 'irri', 'mar'),
 ('AWARE 1.2c', 'Country', 'non', 'irr

### Spatialized LCIA

The `SpatialLCA` class is a subclass of `bw2calc.LCA` that allows for the use of location-specific characterization factors in the characterization matrix of `bw2calc`. `SpatialLCA`can therefore be used as a drop-in replacement for `bw2calc.LCA` in most cases.

In [3]:
# activate the bw project
bw2data.projects.set_current("bw25_ei310")

In [4]:
act = bw2data.Database("ecoinvent-3.10-cutoff").random()
act

'electricity, high voltage, import from US-MRO' (kilowatt hour, CA-SK, None)

In [5]:
method = ('AWARE 1.2c', 'Country', 'unspecified', 'yearly')

In [6]:
LCA = SpatialLCA({act: 1}, method)
LCA.lci()
LCA.lcia()
LCA.score

/Users/romain/anaconda3/envs/bw25/lib/python3.11/site-packages/scikits/umfpack/umfpack.py:736: UmfpackWarning: (almost) singular matrix! (estimated cond. number: 1.21e+13)
  warnings.warn(msg, UmfpackWarning)


0.10860511854635746

In [ ]:
# let's check locations that have been ignored, if any
LCA.ignored_locations

### Generate dataframe of characterization factors used

In [8]:
df = LCA.generate_cf_table()

In [10]:
# we can see under the "CF" column
# the characterization factors used for each exchange in the system
df.head()

,flow,name,reference product,location,amount,CF,impact
0,"(Water, cooling, unspecified natural origin, (...",cast iron production,cast iron,RoW,6.288335e-07,28.180497,1.772084e-05
1,"(Water, cooling, unspecified natural origin, (...","polystyrene production, high impact","polystyrene, high impact",RER,1.138145e-09,17.328016,1.972179e-08
2,"(Water, cooling, unspecified natural origin, (...","cast iron milling, average","cast iron removed by milling, average",RoW,1.305994e-13,28.180497,3.680357e-12
3,"(Water, cooling, unspecified natural origin, (...","electricity production, oil","electricity, high voltage",CA-PE,1.992256e-10,1.713382,3.413496e-10
4,"(Water, cooling, unspecified natural origin, (...","polypropylene production, granulate","polypropylene, granulate",RER,7.043196e-09,17.328016,1.220446e-07


In [40]:
locs = {
    "Auvergne-Rhône-Alpes": "FR-ARA",
    "Nouvelle-Aquitaine": "FR-NA",
    "Occitanie": "FR-OC",
    "Pays de la Loire": "FR-PL",
    "Provence-Alpes-Côte d'Azur": "FR-PACA",
    "Bourgogne-Franche-Comté": "FR-BFC",
    "Bretagne": "FR-BR",
    "Centre-Val de Loire": "FR-CVL",
    "Corse": "FR-CO",
    "Grand Est": "FR-GE",
    "Hauts-de-France": "FR-HF",
    "Île-de-France": "FR-IL",
    "Normandie": "FR-NO"
}

In [33]:
lci = {
    ("h2", f"hydrogen production, from electrolysis {loc}"): {
        "name": "hydrogen production, from electrolysis",
        "reference product": "hydrogen",
        "location": loc,
        "unit": "kilogram",
        "exchanges": [
            {
                "name": "hydrogen production, from electrolysis",
                "reference product": "hydrogen",
                "unit": "kilogram",
                "amount": 1,
                "location": loc,
                "type": "production",
                "input": ("h2", f"hydrogen production, from electrolysis {loc}"),
            },
            {
                "name": "market for electricity, low voltage",
                "reference product": "electricity, low voltage",
                "unit": "kilowatt hour",
                "amount": 55,
                "location": "FR",
                "type": "technosphere",
                "input": ('ecoinvent-3.10-cutoff', '5cb40f535b77005c743a5871a5ef0d2d'),
            },
            {
                "name": "fuel cell production, polymer electrolyte membrane, 2kW electrical, future", # I do not have an electrolyzer dataset at hand!
                "reference product": "fuel cell, polymer electrolyte membrane, 2kW electrical, future",
                "unit": "unit",
                "amount": 1 / 1e6,
                "location": "CH",
                "type": "technosphere",
                "input": ('ecoinvent-3.10-cutoff', '7a473524208226b7397afe6f3193ca95')
            },
            {
                "name": "Water, well, in ground",
                "categories": ("natural resource", "water"),
                "unit": "cubic meter",
                "amount": 12 / 1000, # 12l per kg of hydrogen
                "type": "biosphere",
                "input": ('biosphere', '67c40aae-d403-464d-9649-c12695e43ad8')
            }

        ],
    } for loc in locs.values()
}

In [34]:
lci

{('h2',
  'hydrogen production, from electrolysis FR-ARA'): {'name': 'hydrogen production, from electrolysis', 'reference product': 'hydrogen', 'location': 'FR-ARA', 'unit': 'kilogram', 'exchanges': [{'name': 'hydrogen production, from electrolysis',
    'reference product': 'hydrogen',
    'unit': 'kilogram',
    'amount': 1,
    'location': 'FR-ARA',
    'type': 'production',
    'input': ('h2', 'hydrogen production, from electrolysis FR-ARA')},
   {'name': 'market for electricity, low voltage',
    'reference product': 'electricity, low voltage',
    'unit': 'kilowatt hour',
    'amount': 55,
    'location': 'FR',
    'type': 'technosphere',
    'input': ('ecoinvent-3.10-cutoff', '5cb40f535b77005c743a5871a5ef0d2d')},
   {'name': 'fuel cell production, polymer electrolyte membrane, 2kW electrical, future',
    'reference product': 'fuel cell, polymer electrolyte membrane, 2kW electrical, future',
    'unit': 'unit',
    'amount': 1e-06,
    'location': 'CH',
    'type': 'technosphere

In [32]:
[act.key for act in bw2data.Database("biosphere") if act["name"] == "Water, well, in ground" and act["categories"]]


[('biosphere', '67c40aae-d403-464d-9649-c12695e43ad8')]

In [35]:
bw2data.Database("h2").write(lci)

19:01:28 [warning  ] Not able to determine geocollections for all datasets. This database is not ready for regionalization.


100%|██████████| 13/13 [00:00<00:00, 9072.54it/s]

19:01:28 [info     ] Vacuuming database            


In [36]:
bw2data.databases

Databases dictionary with 4 object(s):
	biosphere
	ecoinvent-3.10-cutoff
	h2
	hydrogen production

In [41]:
for long, short in locs.items():
    act = [act for act in bw2data.Database("h2") if act["location"] == short][0]
    LCA = SpatialLCA({act: 1}, method)
    LCA.lci()
    LCA.lcia()
    print(f"Location: {long}, score: {LCA.score}")


/Users/romain/anaconda3/envs/bw25/lib/python3.11/site-packages/scikits/umfpack/umfpack.py:736: UmfpackWarning: (almost) singular matrix! (estimated cond. number: 1.21e+13)
  warnings.warn(msg, UmfpackWarning)


Location: Auvergne-Rhône-Alpes, score: 2.0619179340947085
Location: Nouvelle-Aquitaine, score: 2.138260028210425
Location: Occitanie, score: 2.2240827194323174
Location: Pays de la Loire, score: 2.2328988084691446
Location: Provence-Alpes-Côte d'Azur, score: 2.082538160997956
Location: Bourgogne-Franche-Comté, score: 2.066881737953736
Location: Bretagne, score: 2.174215243358958
Location: Centre-Val de Loire, score: 2.0902908340972317
Location: Corse, score: 2.5936978846452803
Location: Grand Est, score: 2.0600159127661755
Location: Hauts-de-France, score: 2.0645213910092934
Location: Île-de-France, score: 2.0749227122560114
Location: Normandie, score: 2.070808196715916


In [42]:
LCA.generate_cf_table()

,flow,name,reference product,location,amount,CF,impact
0,"(Water, cooling, unspecified natural origin, (...",cast iron production,cast iron,RoW,2.510268e-05,28.180497,7.074059e-04
1,"(Water, cooling, unspecified natural origin, (...","polystyrene production, high impact","polystyrene, high impact",RER,4.439220e-06,17.328016,7.692287e-05
2,"(Water, cooling, unspecified natural origin, (...","cast iron milling, average","cast iron removed by milling, average",RoW,2.704166e-10,28.180497,7.620473e-09
3,"(Water, cooling, unspecified natural origin, (...","electricity production, oil","electricity, high voltage",CA-PE,5.875734e-09,1.713382,1.006738e-08
4,"(Water, cooling, unspecified natural origin, (...","polypropylene production, granulate","polypropylene, granulate",RER,3.241600e-07,17.328016,5.617050e-06
...,...,...,...,...,...,...,...
8875,"(Water, (water, surface water))","petroleum and gas production, onshore","natural gas, high pressure",TH,7.179715e-09,-6.981635,-5.012615e-08
8876,"(Water, (water, surface water))","treatment of waste emulsion paint, municipal i...",waste emulsion paint,CH,1.551506e-10,-0.965402,-1.497827e-10
8877,"(Water, (water, surface water))","aluminium production, primary, liquid, prebake","aluminium, primary, liquid","IAI Area, Russia & RER w/o EU27 & EFTA",1.141075e-06,-12.226962,-1.395188e-05
8878,"(Water, (water, surface water))",citric acid production,citric acid,RNA,6.676624e-09,-27.325808,-1.824441e-07
